<a href="https://colab.research.google.com/github/SRARNAB7/HDS_5230_07_Arnab/blob/main/Week%2010/stream_service_data_science_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install fastapi uvicorn

In [6]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score
import fastapi
from typing import Optional

In [7]:


_x = [1]
_y = [1]
_prediction = ["True"]
_result = ["True"]
model = DummyClassifier(strategy="constant", constant="True")
model.fit(np.array([_x,_y]).reshape(1, -1), _result)


DummyClassifier(constant='True', strategy='constant')

In [15]:
def model_evaluation(y_test, y_pred) :
    print('Trained Model Test Data Accuracy Score :',accuracy_score(y_test, y_pred)*100)
    testacc=accuracy_score(y_test, y_pred)
   # testrecall=recall_score(y_test, y_pred, pos_label="True")
   # testprecision=precision_score(y_test, y_pred, pos_label="True")
    testrecall = recall_score(y_test, y_pred, pos_label='True', zero_division=0)
    testprecision = precision_score(y_test, y_pred, pos_label='True', zero_division=0)
    print(' ')
    print(classification_report(y_test, y_pred))
    return testacc, testrecall, testprecision

In [16]:
def create_eval_df(x, y, result) :
    data = {"x": x, "y": y, "result": result}
    df = pd.DataFrame(data)
    eval_df = df.copy()
    eval_df["predict"] = model.predict(data[x,y])
    eval_df["correct"] = eval_df["predict"] == eval_df["result"]
    eval_df["correctandtrue"] = eval_df["predict"].isin([True]) & eval_df["result"].isin([True])
    return eval_df

In [17]:
app = fastapi.FastAPI()

@app.get("/predict/")
def predict(id:int = 0 , x:float =0, y:float=0):
    print(id, x, y)
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([x,y]).reshape(1, -1))
    _prediction.insert(id, prediction[0])
    print(prediction[0])
    return dict([("prediction", prediction[0])])

@app.get("/result/")
def result(id:int, result:str):
    print(result)
    _result.insert(id, result)
    return dict([("message", "Copied result for {}".format(id))])

@app.get("/retrain/")
def retrain(last_n:int):
    print ("RETRAINING over last {}".format(last_n))
    #print(_x)
    #print(_y)
    #X = np.rec.fromarrays([_x, _y])[:(-1*last_n)]
    #Y = np.rec.fromarrays([_result])[:(-1*last_n)]
    X = np.array(list(zip(_x[-last_n:], _y[-last_n:])))
    Y = np.array(_result[-last_n:])
    #print(X, Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40)

    print(X_train)
    print(y_train)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)

    return {"message": "Model retrained", "accuracy": testacc, "recall": testrecall, "precision": testprecision}

@app.get("/new_model/")
def new_model(model:str, last_n:int, strategy:Optional[str] = None, k:Optional[int] = 0):
    print(model, last_n, strategy, k)
    if model == "Dummy" :
        model = DummyClassifier(strategy=strategy)
    elif model == "KNN" :
        model = KNeighborsClassifier(n_neighbors=k)
    #model.set_params(model_params)

    message = retrain(last_n=last_n)

    returns = {"message": "Model trained {}".format(model), "train": message}
    print(returns)

    return returns

# @app.get("/evaluate_model/")
# def evaluate_model(last_n:int):
#    return {"message": "Item created", "last_n": last_n}

In [18]:
for i in range(100):
    _x.append(i)
    _y.append(i)
    _result.append("True" if i % 2 == 0 else "False")

In [19]:
#print(predict(1, 1, 1))
#result(1, True)
new_model("Dummy", 50, "stratified")

Dummy 50 stratified 0
RETRAINING over last 50
[[84 84]
 [72 72]
 [60 60]
 [67 67]
 [68 68]
 [88 88]
 [75 75]
 [85 85]
 [94 94]
 [50 50]
 [62 62]
 [54 54]
 [74 74]
 [70 70]
 [86 86]
 [59 59]
 [51 51]
 [57 57]
 [76 76]
 [89 89]
 [64 64]
 [97 97]
 [79 79]
 [87 87]
 [77 77]
 [93 93]
 [65 65]
 [82 82]
 [58 58]
 [55 55]]
['True' 'True' 'True' 'False' 'True' 'True' 'False' 'False' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'False' 'False' 'False' 'True' 'False'
 'True' 'False' 'False' 'False' 'False' 'False' 'False' 'True' 'True'
 'False']
Trained Model Test Data Accuracy Score : 45.0
 
              precision    recall  f1-score   support

       False       0.00      0.00      0.00        11
        True       0.45      1.00      0.62         9

    accuracy                           0.45        20
   macro avg       0.23      0.50      0.31        20
weighted avg       0.20      0.45      0.28        20

{'message': "Model trained DummyClassifier(strategy='stratified')", 'train': {'

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'message': "Model trained DummyClassifier(strategy='stratified')",
 'train': {'message': 'Model retrained',
  'accuracy': 0.45,
  'recall': 1.0,
  'precision': 0.45}}

In [20]:
new_model("KNN", last_n=50, k=3)

KNN 50 None 3
RETRAINING over last 50
[[73 73]
 [57 57]
 [52 52]
 [79 79]
 [78 78]
 [80 80]
 [62 62]
 [91 91]
 [99 99]
 [56 56]
 [97 97]
 [63 63]
 [90 90]
 [61 61]
 [50 50]
 [59 59]
 [67 67]
 [92 92]
 [87 87]
 [84 84]
 [89 89]
 [93 93]
 [65 65]
 [72 72]
 [76 76]
 [96 96]
 [98 98]
 [68 68]
 [71 71]
 [86 86]]
['False' 'False' 'True' 'False' 'True' 'True' 'True' 'False' 'False'
 'True' 'False' 'False' 'True' 'False' 'True' 'False' 'False' 'True'
 'False' 'True' 'False' 'False' 'False' 'True' 'True' 'True' 'True' 'True'
 'False' 'True']
Trained Model Test Data Accuracy Score : 50.0
 
              precision    recall  f1-score   support

       False       0.00      0.00      0.00        10
        True       0.50      1.00      0.67        10

    accuracy                           0.50        20
   macro avg       0.25      0.50      0.33        20
weighted avg       0.25      0.50      0.33        20

{'message': 'Model trained KNeighborsClassifier(n_neighbors=3)', 'train': {'message': 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'message': 'Model trained KNeighborsClassifier(n_neighbors=3)',
 'train': {'message': 'Model retrained',
  'accuracy': 0.5,
  'recall': 1.0,
  'precision': 0.5}}

In [32]:
@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

In [33]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())